<a href="https://colab.research.google.com/github/cvanessaribeiro/pospucrj_sprint_analisededados/blob/main/Notebook_Sprint_Analisededadoseboaspraticas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**# MVP de Análise de Dados e Boas Práticas**

# MVP de Análise de Dados e Boas Práticas

## Professores Tatiana Escovedo e Hugo Villamizar
## Aluna Carla Vanessa Florentino Ribeiro

## 1. Definição do Problema

O dataset usado neste projeto será o **dataset_vendas**, oriundo da exportação da base de vendas de empresa de telecom. Seu objetivo é analisar a performance das vendas por vendedor, considerando as quebras durante todo o processo, afim de identificar o vendedor que possui a estratégia mais qualitativa.

**Restrições do dataset:**
- Trata-se de um dataset com vendas de contratos de banda larga nos estados RJ, SP, MG, ES e DF, exportada do sistema ERP de um a empresa do segmento de telecom;
- O período analisado é jun/23 a nov/23;
- **Este não é um dataset de machine learning.**

**Atributos do dataset:**
1. **id_venda** - número do contrato da venda
2. **nome_vendedor** - nome do vendedor
3. **nome_cliente** - nome do cliente
4. **uf_cidade** - estado e cidade do cliente
5. **data_criacao** - data da venda
6. **status_contrato** - status atual do contrato (Em Ativação, Cancelado Antes Ativação, Ativado e Churn)
7. **status_fatura** - status atual da primeira fatura (A Vencer, Paga, Vencida)

**Regras de Negócio:**
*   Etapa Contrato Em Ativacao: Venda na fila de Ativaçao;
*   Etapa Contrato Cancelado Antes Ativacao: Venda cancelada antes da ativação (**1ª QUEBRA**);
*   Etapa Contrato Ativado: Venda Ativada;
*   Etapa Contrato Churn: Venda Cancelada Após a Ativação (**2ª QUEBRA**);
*   Etapa Status Fatura Vencida: Venda instalada e fatura não paga (**3ª QUEBRA**)
*   **Venda Bruta**: contagem de todos os contratos, independente do status e da fatura;


In [8]:
# Imports
import pandas as pd #Biblioteca responsável para fazer a leitura e impressão de base de dados

In [12]:
# Usando base do Github
df = pd.read_csv('https://github.com/cvanessaribeiro/pospucrj_sprint_analisededados/blob/main/dataset_vendas.csv',encoding='utf-8' )

ParserError: ignored

## 2. Carga de Dados

texto explicativo

In [7]:
# Carregamento arquivo xlsx usando URL do Github

# URL importação
url = "https://github.com/cvanessaribeiro/pospucrj_sprint_analisededados/blob/main/dataset_vendas.csv"

# Informar cabeçalho das colunas
colunas = ['id_venda', 'vendedor', 'cliente', 'uf_cidade', 'data_criacao', 'status_contrato', 'status_fatura']

# Lê o arquivo utilizando as colunas informadas
dataset = pd.dataset_vendas_xlsx(url, names=colunas, skiprows=1, delimiter=',')


NameError: ignored

## 3. Análise de Dados

### 3.1. Estatísticas Descritiva

### 3.2. Visualizações

## 4. Pré-Processamento de dados

## Conclusão